<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/APE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our own implementation of APE from this [blogpost](https://towardsdatascience.com/automated-prompt-engineering-the-definitive-hands-on-guide-1476c8cd3c50)

In [ ]:
!pip install -qU langchain --progress-bar off
!pip install -qU langchainhub --progress-bar off
!pip install -qU langchain_community --progress-bar off
!pip install -qU langchain-fireworks --progress-bar off
!pip install -qU fireworks-ai --progress-bar off
!pip install -qU datasets --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
# @title load fireworks API key
#connect to google drive
from google.colab import drive
import json
import os


drive.mount('/content/drive')

with open('/content/drive/MyDrive/env/env.json') as jsonfile:
    env = json.load(jsonfile)

os.environ["FIREWORKS_API_KEY"] = env['fireworks.ai']['apiKey']


Mounted at /content/drive


In [ ]:
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
# Turn this on only if you want to debug other wise it's hard to see the conversations.
set_debug(True)

In [ ]:
from langchain_fireworks import Fireworks
llm = Fireworks(
  model="accounts/fireworks/models/mixtral-8x7b-instruct",
  fireworks_api_key=env['fireworks.ai']['apiKey'],
  base_url="https://api.fireworks.ai/inference/v1/completions",
  temperature= 0, max_tokens= 1040
)

optimizer_llm = Fireworks(
  model="accounts/fireworks/models/mixtral-8x22b-instruct",
  fireworks_api_key=env['fireworks.ai']['apiKey'],
  base_url="https://api.fireworks.ai/inference/v1/completions",
  temperature= 0, max_tokens= 1040
)

reviewer_llm = Fireworks(
  model="accounts/fireworks/models/mixtral-8x7b-instruct",
  fireworks_api_key=env['fireworks.ai']['apiKey'],
  base_url="https://api.fireworks.ai/inference/v1/completions",
  temperature= 0, max_tokens= 20
)



In [ ]:
# @title Load our dataset
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("lukaemon/bbh", "geometric_shapes", cache_dir="./bbh_nshapes_cache")
data = dataset["test"]
data = data.shuffle(seed=1234)

training = data.select(range(100))
df_train = pd.DataFrame({"question": training["input"], "answer": training["target"]})

test = data.select(range(100, 200))
df_test = pd.DataFrame({"question": test["input"], "answer": test["target"]})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
df_train['question'][0], df_train['answer'][0],

('This SVG path element <path d="M 50.00,45.00 L 77.00,45.00 L 77.00,33.00 L 50.00,33.00 L 50.00,45.00"/> draws a\nOptions:\n(A) circle\n(B) heptagon\n(C) hexagon\n(D) kite\n(E) line\n(F) octagon\n(G) pentagon\n(H) rectangle\n(I) sector\n(J) triangle\n(K) trapezoid',
 '(K)')

In [ ]:
# @title Check response without prompt optimization (wrong answer)
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(template=f"""
<s>[INST] Solve the given problem about geometric shapes.

{df_train['question'][0]}
[/INST]
""")


chain = prompt | llm
chain.invoke({})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:Fireworks] Entering LLM run with input:
{
  "prompts": [
    "<s>[INST] Solve the given problem about geometric shapes.\n\nThis SVG path element <path d=\"M 50.00,45.00 L 77.00,45.00 L 77.00,33.00 L 50.00,33.00 L 50.00,45.00\"/> draws a\nOptions:\n(A) circle\n(B) heptagon\n(C) hexagon\n(D) kite\n(E) line\n(F) octagon\n(G) pentagon\n(H) rectangle\n(I) sector\n(J) triangle\n(K) trapezoid\n[/INST]"
  ]
}
[llm/end] [chain:RunnableSequence > llm:Fireworks] [1.62s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The given SVG path represents a rectangle. The \"M\" command moves the virtual pen to a starting point, and the following pairs of \"L\"

'The given SVG path represents a rectangle. The "M" command moves the virtual pen to a starting point, and the following pairs of "L" commands draw the lines that form the rectangle.\n\nSo, the answer is (H) rectangle.'

In [ ]:
# @title Prompts

REVIEW_PROMPT = """
You are a review model tasked with evaluating the correctness of a response to a navigation problem.
The response may contain detailed steps and explanations, but the final answer is the key point.
Please determine if the final answer provided in the response is correct based on the ground truth number.
Respond with 'True' if the final answer is correct and 'False' if it is not.
Only respond with 'True' or 'False', nothing else.

Model Response:
{model_response}

Ground Truth:
{ground_truth}"""

META_PROMPT = """
<EXPLANATION>
I have some prompts along with their corresponding accuracies.
The prompts are arranged in ascending order based on their accuracy, where higher accuracy indicate better quality.
</EXPLANATION>

<PROMPTS>
{prompt_scores}
</PROMPTS>

Each prompt was used together with a problem statement around geometric shapes.

<EXAMPLE>
<QUESTION>
This SVG path element <path d="M 55.57,80.69 L 57.38,65.80 M 57.38,65.80 L 48.90,57.46 M 48.90,57.46 L 45.58,47.78 M 45.58,47.78 L 53.25,36.07 L 66.29,48.90 L 78.69,61.09 L 55.57,80.69"/> draws a Options: (A) circle (B) heptagon (C) hexagon (D) kite (E) line (F) octagon (G) pentagon (H) rectangle (I) sector (J) triangle
</QUESTION>
<ANSWER>
(B)
</ANSWER>
</EXAMPLE>

<TASK>
Write a new prompt that will achieve an accuracy as high as possible and that is different from the old ones.
</TASK>


<RULES>
- It is very important that the new prompt is distinct from ALL the old ones!
- Ensure that you analyse the prompts with a high accuracy and reuse the patterns that worked in the past
- Ensure that you analyse the prompts with a low accuracy and avoid the patterns that didn't worked in the past
- Think out loud before creating the prompt. Describe what has worked in the past and what hasn't. Only then create the new prompt.
- Use all available information like prompt length, formal/informal use of language, etc for your analysis.
- Be creative, try out different ways of prompting the model. You may even come up with hypothetical scenarios that might improve the accuracy.
- You are generating system prompts. This means that there should be no placeholders in the prompt, as they cannot be filled at runtime. Instead focus on general instructions that will help the model to solve the task.
- Write your new prompt in double square brackets. Use only plain text for the prompt text and do not add any markdown (i.e. no hashtags, backticks, quotes, etc).
</RULES>
"""

In [ ]:
# @title Helpers

import re

def generate_response(prompt, use_llm=None):
  prompt = f"<s>[INST] {prompt}[/INST]"
  callable_llm = llm if use_llm is None else use_llm
  chain = PromptTemplate(template=prompt) | callable_llm
  return chain.invoke({})


def generate_and_review(row, prompt):
  answer_prompt = f"{prompt}\n{row['question']}"
  model_response = generate_response(answer_prompt)

  review_prompt = REVIEW_PROMPT.format(model_response=model_response, ground_truth=row['answer'])
  is_correct = review_response = generate_response(review_prompt)
  return is_correct, model_response

def prepare_prompt_accuracies_for_meta_prompt(prompt_accuracies):
    sorted_prompts_string = ""
    sorted_prompts = sorted(prompt_accuracies, key=lambda d: d['accuracy'])

    print(sorted_prompts)

    for prompt in sorted_prompts:
      s = f"<PROMPT>\n<PROMPT_TEXT>\n{prompt['prompt']}\n</PROMPT_TEXT>\n<ACCURACY>\nAccuracy: {prompt['accuracy']}\n</ACCURACY>\n</PROMPT>\n\n"
      sorted_prompts_string += s

    return sorted_prompts_string

def extract_optimizer_suggestion(text):
    # the optimizer LLM produce [[ output]]
    # Use regex to find the sentence wrapped with [[]]
    # E.g.

    # <ANALYSIS>
    # The prompt with the highest accuracy is not provided, so I cannot directly compare the patterns that worked in the past. However, I can still analyze the given prompt and its accuracy.

    # The given prompt "Solve the given problem about geometric shapes." has an accuracy of 0. This indicates that the prompt is too broad and does not provide enough context or guidance for the model to accurately solve the problem.

    # To improve the accuracy, I will create a more specific and detailed prompt that provides clear instructions and context. I will also avoid using overly broad or vague language.
    # </ANALYSIS>

    # [[Analyze the given SVG path element and determine the shape it represents. Choose the correct option from the provided list.]]
    pattern = r'\[\[(.*?)\]\]'
    result = re.search(pattern, text, re.DOTALL)

    if result:
        return result.group(1)
    else:
        return None


In [ ]:
INITIAL_PROMPT = """
Solve the given problem about geometric shapes.
"""
def train(training_data, prompt=None):
  for i, row in training_data.iterrows():
    (is_correct, model_response) = generate_and_review(row, prompt=prompt)
    training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
    training_data.loc[i, 'model_response'] = model_response

  overall_accuracy = sum(training_data["is_correct"]) / len(training_data)
  return overall_accuracy, training_data

def ape(initial_prompt=INITIAL_PROMPT, steps = 5, training_length = 5):
  prompt_accuracies = []

  for i in range(steps + 1):
    if i == 0:
      new_prompt = INITIAL_PROMPT
    else:
      # [{
      #     "prompt": "Solve the given problem about geometric shapes.",
      #     "accuracy": 0
      # }]
      prompt_scores= prepare_prompt_accuracies_for_meta_prompt(prompt_accuracies=prompt_accuracies)
      meta_prompt=META_PROMPT.format(prompt_scores=prompt_scores)
      response = generate_response(prompt=meta_prompt, use_llm=optimizer_llm)
      new_prompt = extract_optimizer_suggestion(response)


    loss, training_result = train(training_data=df_train.loc[:training_length], prompt=new_prompt)
    prompt_accuracies.append({
        "prompt": new_prompt,
        "accuracy": loss
    })

  return prompt_accuracies


result = ape(initial_prompt=INITIAL_PROMPT, steps=5, training_length=5)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[chain/end] [chain:RunnableSequence] s] Exiting Chain run with output:
{
  "output": " (H) rectangle\n\nThe given SVG path element represents a rectangle. The \"M\" command sets the starting point, and the subsequent \"L\" commands define the lines that make up the rectangle. It starts at (50,45), goes to (77,45), then down to (77,33), then across to (50,33), and finally back to the starting point (50,45)."
}
[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[chain/end] [chain:RunnableSequen

<ipython-input-85-533c2d148d80>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
<ipython-input-85-533c2d148d80>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
<ipython-input-85-533c2d148d80>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

[llm/end] [chain:RunnableSequence > llm:Fireworks] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " [J, Triangle]",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RunnableSequence] s] Exiting Chain run with output:
{
  "output": " [J, Triangle]"
}
[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:Fireworks] Entering LLM run with input:
{
  "prompts": [
    "<s>[INST] \nYou are a review model tasked with evaluating the correctness of a response to a navigation problem. \nThe response may contain detailed steps and explanations, but the final answer is the key point. \nPlease determine if the final 

<ipython-input-85-533c2d148d80>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
<ipython-input-85-533c2d148d80>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
<ipython-input-85-533c2d148d80>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

[llm/end] [chain:RunnableSequence > llm:Fireworks] [2.34s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " (B) heptagon\n\nThe \"M 64.00,63.00\" command moves the virtual pen to the point at (64, 63). The following \"L\" commands draw lines from the current point to the specified points. The sequence of points (64, 63), (44, 63), (44, 50), (64, 50), (83, 57), (64, 76), and back to (64, 63) forms a heptagon (7-sided polygon).",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RunnableSequence] [2.34s] Exiting Chain run with output:
{
  "output": " (B) heptagon\n\nThe \"M 64.00,63.00\" command moves the virtual pen to the point at (64, 63). The following \"L\" commands draw lines from the current point to the specified points. The sequence of points (64, 63), (44, 63), (44, 50), (64, 50), (83, 57), (64, 76), and back to (64, 63) forms a heptagon (7-sided polygon)."
}

...

[llm/end] [chain:RunnableSequence > llm:Fireworks] [2.92s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The correct answer is (D) kite. This path forms a kite shape because it consists of four line segments that meet at their endpoints to create an asymmetrical, quadrilateral figure. The first two line segments (M 64.00,63.00 L 44.00,63.00) form a horizontal line, followed by another horizontal line (L 44.00,50.00 L 64.00,50.00). The third line segment (L 64.00,50.00 L 64.00,37.00) is a vertical line, and the last line segment (L 64.00,37.00 L 83.00,57.00) forms a diagonal line. The last point (L 83.00,57.00) then connects back to the starting point (L 64.00,63.00) to complete the kite shape.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RunnableSequence] [2.92s] Exiting Chain run with output:
{
  "output": " The correct answer is (D) kite. This path forms 

<ipython-input-85-533c2d148d80>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
<ipython-input-85-533c2d148d80>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
<ipython-input-85-533c2d148d80>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

[llm/end] [chain:RunnableSequence > llm:Fireworks] [7.24s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " Let's break down the SVG path element together!\n\n<path d=\"M 64.00,63.00 <!-- Move to the starting point: (64, 63) -->\nL 44.00,63.00 <!-- Draw a line to: (44, 63) -->\nL 44.00,50.00 <!-- Draw a line to: (44, 50) -->\nL 64.00,50.00 <!-- Draw a line to: (64, 50) -->\nL 64.00,37.00 <!-- Draw a line to: (64, 37) -->\nL 83.00,57.00 <!-- Draw a line to: (83, 57) -->\nL 64.00,76.00 <!-- Draw a line to: (64, 76) -->\nL 64.00,63.00 <!-- Draw a line back to the starting point: (64, 63) -->\n\"/>\n\nThe path consists of several connected line segments. Let's identify the endpoints of each segment:\n\n(64, 63) - (44, 63) - (44, 50) - (64, 50) - (64, 37) - (83, 57) - (64, 76) - (64, 63)\n\nNow, let's find the differences between the x-coordinates and y-coordinates of each consecutive pair of points:\n\nΔx: -20, 0, -20, 27, -19, -26, 0\nΔy: 0, -13, -20, 17, 

<ipython-input-85-533c2d148d80>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
<ipython-input-85-533c2d148d80>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  training_data.loc[i, 'is_correct'] = is_correct.strip() == 'True'
<ipython-input-85-533c2d148d80>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

[llm/end] [chain:RunnableSequence > llm:Fireworks] [2.35s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The given SVG path element represents a shape with straight lines, so we can eliminate options that involve curves such as a circle or a sector (option A and I). We can also eliminate line (option E) because it only has two points, while our path has many more.\n\nNow, let's count the number of sides or corners in the shape. Starting from the first point (64, 63), the path moves to (44, 63), (44, 50), (64, 50), (64, 37), (83, 57), (64, 76), and back to (64, 63). That makes a total of 7 different points, meaning our shape has 7 sides or corners.\n\nTherefore, the correct answer is (B) heptagon.",
        "generation_info": null,
        "type": "Generation"
      }
    ]
  ],
  "llm_output": null,
  "run": null
}
[chain/end] [chain:RunnableSequence] [2.36s] Exiting Chain run with output:
{
  "output": " The given SVG path element represents a shape

In [ ]:
text="\n\n<NEW_PROMPT>\n[[\nAnalyze the given SVG path element and determine the geometric shape it represents.\n]]\n</NEW_PROMPT>"
extract_optimizer_suggestion(text)

'\nAnalyze the given SVG path element and determine the geometric shape it represents.\n'

In [ ]:
result

[{'prompt': '\nSolve the given problem about geometric shapes.\n',
  'accuracy': 0.0},
 {'prompt': '\n\nAnalyze the given SVG path element and determine the geometric shape it represents.\n\n',
  'accuracy': 0.16666666666666666},
 {'prompt': '\n\nInvestigate the given SVG path element and deduce the geometric shape it forms. Make it a fun challenge!\n\n',
  'accuracy': 0.16666666666666666},
 {'prompt': '\n\nExamine the provided SVG path element and unravel the mystery of the geometric shape it forms. Let your imagination run wild as you decipher the code!\n\n',
  'accuracy': 0.0},
 {'prompt': '\n\nDecipher the given SVG path element and determine the geometric shape it forms. Make it a fun and engaging challenge!\n\n',
  'accuracy': 0.16666666666666666},
 {'prompt': '\n\nDelve into the given SVG path element and uncover the mystery of the geometric shape it represents. Use your analytical skills to decipher the code and make an informed decision.\n\n',
  'accuracy': 0.5}]